# Imports

In [1]:
# Bibliotecas para aumentar dataset #####################
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from keras.applications.vgg16 import VGG16 #Biblioteca cnn-vgg16
import numpy as np

#####################  Bibliotecas para separar dados #####################
import cv2 #Trabalhar com imagens
from sklearn.model_selection import train_test_split #Separar dados de treino e teste
from sklearn.utils import shuffle #Embaralhar dados
import os #sistema operacional

# Função para aumentar o dataset

In [2]:
def dataAugmentation(path, classe):
    image = load_img(path)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    # contruindo o gerador de imagem para data augmentation
    aug = ImageDataGenerator(
        rotation_range=30,
        zoom_range=0.15,
        #width_shift_range=0.2,
        #height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode="nearest")
    
    total = 0
    saveDir = ''
    #local de destino das imagens
    if(classe == 0):
        saveDir = os.getcwd()+'/Aug/NewG0/'
    elif(classe == 1):
        saveDir = os.getcwd()+'/Aug/NewG1/'
    elif(classe == 2):
        saveDir = os.getcwd()+'/Aug/NewG2/'
    elif(classe == 3):
        saveDir = os.getcwd()+'/Aug/NewG3/'
    elif(classe == 4):
        saveDir = os.getcwd()+'/Aug/NewG4/'
        
    imageGen = aug.flow(image, batch_size=1, save_to_dir=saveDir,
            save_prefix="image", save_format="jpg")
    # loop que faz o controle do número de imagens a serem geradas (no caso 100)
    for image in imageGen:
        if total == 25:
            break
        total += 1

# Data augmentation

In [3]:
path = os.getcwd()+'/GrausPt/'

for caminho, d, file in os.walk(path):
    for filename in file:
        img = cv2.imread(os.path.join(caminho,filename))

        if caminho == os.getcwd()+'/GrausPt/g0':
            cv2.imwrite(os.getcwd()+'/Aug/NewG0/'+filename, img)
            dataAugmentation(os.path.join(caminho,filename), 0)
       
        elif caminho == os.getcwd()+'/GrausPt/g1':
            cv2.imwrite(os.getcwd()+'/Aug/NewG1/'+filename, img)
            dataAugmentation(os.path.join(caminho,filename), 1)
          
        elif caminho == os.getcwd()+'/GrausPt/g2':
            cv2.imwrite(os.getcwd()+'/Aug/NewG2/'+filename, img)
            dataAugmentation(os.path.join(caminho,filename), 2)
            
        elif caminho == os.getcwd()+'/GrausPt/g3':
            cv2.imwrite(os.getcwd()+'/Aug/NewG3/'+filename, img)
            dataAugmentation(os.path.join(caminho,filename), 3)
        
        elif caminho == os.getcwd()+'/GrausPt/g4':
            cv2.imwrite(os.getcwd()+'/Aug/NewG4/'+filename, img)
            dataAugmentation(os.path.join(caminho,filename), 4)

# Carregar imagens

In [ ]:
path = os.getcwd()+'/Aug/'
data_set = []
saidas = []
        
for caminho, d, file in os.walk(path):
    for filename in file:
        
        img = cv2.imread(os.path.join(caminho,filename))
        
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Deixa a imagem com tons de cinza
        #equalized_image = cv2.equalizeHist(gray) #Aumentar o contraste da imagem
        img = np.array(cv2.resize(img, (128,128), interpolation = cv2.INTER_AREA))
        data_set.append(img)
        
        if caminho == path+'NewG0':
            saidas.append(0)
        elif caminho == path+'NewG1':
            saidas.append(1)
        elif caminho == path+'NewG2':
            saidas.append(2)
        elif caminho == path+'NewG3':
            saidas.append(3)
        elif caminho == path+'NewG4':
            saidas.append(4)

In [ ]:
len(saidas), len(data_set)

In [ ]:
X, Y = shuffle(data_set, saidas) #embaralha os dados

train_X = []
train_Y = []

test_X = []
test_Y = []

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, train_size=0.8, random_state=42) #separa os dados em treino e teste com porcentagen de 80% para treino

#Deixando no mesmo tipo do array do data_set
train_Y = np.array(train_Y).astype('uint8')
test_Y = np.array(test_Y).astype('uint8')

train_X = np.array(train_X)
test_X = np.array(test_X)

In [ ]:
from keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

print('Tamanho e quantidade da base de dados de treino:', train_X.shape, train_Y.shape)

print('Tamanho e quantidade da base de dados de teste:', test_X.shape, test_Y.shape)

In [ ]:
# Encontra o numero de classes
classes = np.unique(train_Y) #Pega o valores sem repetição
nClasses = len(classes)
print('Numero total de classes:', nClasses)
print('Saida das classes:', classes)

In [ ]:
import cv2
plt.figure(figsize=[5,5])

#Mostra a primeira imagem dos dados de treino

plt.subplot(121)
img = cv2.cvtColor(train_X[0,:,:],cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title("Classe : {}".format(train_Y[0]))

#Mostra a primeira imagem dos dados de teste
plt.subplot(122)
img = cv2.cvtColor(train_X[5,:,:],cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title("Classe : {}".format(test_Y[5]))

In [ ]:
# Deixar as imagens na mesma dimensão
train_X = train_X.reshape(-1, 128, 128, 3)
test_X = test_X.reshape(-1, 128, 128, 3)
train_X.shape, test_X.shape

In [ ]:
#Normalizando a imagens deixando os pixels entre 0 e 1
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

In [ ]:
# Mudança de codificação de código de classe para se padronizar com o Keras.
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Testando a padronização de rotulo
print('Rotulo Original:', train_Y[0])
print('Depois da conversão para o one-hot:', train_Y_one_hot[0])

In [ ]:
#Validação do modelo
from sklearn.model_selection import train_test_split
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)

In [ ]:
train_X[0].shape, valid_X.shape, train_label.shape, valid_label.shape

In [ ]:
#A rede
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD

In [ ]:
batch_size = 64 #Quantidade de neuronios que seram utilizados.
epochs = 10 #Interações
num_classes = 5

# TRANSFER LEARNING

In [ ]:
vgg = keras.applications.VGG16(input_shape=(128,128,3), include_top = False, weights= 'imagenet')

In [ ]:
x = vgg.output
x = Flatten()(x)
x = Dense(3078,activation='relu')(x) 
x = Dropout(0.5)(x)
x = Dense(256,activation='relu')(x) 
x = Dropout(0.2)(x)
out = Dense(5,activation='softmax')(x)
tf_model=Model(inputs=vgg.input,outputs=out)
for layer in tf_model.layers[:20]:
    layer.trainable=False

In [ ]:
tf_model.compile(optimizer=keras.optimizers.Adam(), loss = keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
train_label.shape

In [ ]:
history = tf_model.fit(train_X, train_label, batch_size = 64, epochs = 60, initial_epoch = 0, validation_data = (valid_X, valid_label))

In [ ]:
#Salvando treinamento
tf_model.save("fashion_model_dropout.h5py")

In [ ]:
#Validação da parte de teste
test_eval = tf_model.evaluate(test_X, test_Y_one_hot, verbose=1)

In [ ]:
#Plotando graficos referentes a treinamento e validação acurácia e loss
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
#Predizendo a  classe a partir da base de teste_X
predicted_classes = tf_model.predict(test_X)
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)
predicted_classes.shape, test_Y.shape

In [ ]:
correct = np.where(predicted_classes==test_Y)[0]

In [ ]:
#Mostrando a quantidade de acertos e alguns exemplos
correct = np.where(predicted_classes==test_Y)[0]
print ("Found %d correct labels" % len(correct))
for i, correct in enumerate(correct[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[correct].reshape(128,128,3), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], test_Y[correct]))
    plt.tight_layout()

In [ ]:
#Mostrando a quantidade de erros e alguns exemplos
incorrect = np.where(predicted_classes!=test_Y)[0]
print ("Found %d incorrect labels" % len(incorrect))
for i, incorrect in enumerate(incorrect[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_X[incorrect].reshape(128,128,3), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], test_Y[incorrect]))
    plt.tight_layout()

In [ ]:
#Precisão de classificação
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(test_Y, predicted_classes, target_names=target_names))

# REDE CRIADA PARA CLASSIFICAR

In [ ]:
fashion_model = Sequential()

fashion_model.add(Conv2D(16, kernel_size=(3, 3),activation='relu',padding='same',input_shape=(128,128,3)))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Dropout(0.2))

fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.01))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.01))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.01))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.25))

fashion_model.add(Flatten())
fashion_model.add(Dense(512, activation='sigmoid'))
fashion_model.add(LeakyReLU(alpha=0.01))           
fashion_model.add(Dropout(0.5))
fashion_model.add(Dense(num_classes, activation='softmax'))
sgd = SGD(lr = 0.1, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
fashion_model.summary()

In [ ]:
#Compilando a rede neural criada usando como metrica a accurancy
fashion_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
import time

In [ ]:
ini = time.time()
#Treinando a rede
fashion_train_dropout = fashion_model.fit(train_X, train_label, batch_size=64,epochs=100,verbose=1,validation_data=(valid_X, valid_label))
fim = time.time()
print(f'tempo: {ini-fim}')

In [ ]:
import os
beep = lambda x: os.system("echo -n '\a';sleep 0.2;" * x)
beep(10)

In [ ]:
#Salvando treinamento
fashion_model.save("semTransfer.h5py")

In [ ]:
#Validação da parte de teste
test_eval = fashion_model.evaluate(test_X, test_Y_one_hot, verbose=1)

### carregar modelo treinado

In [ ]:
#carregar modelo salvo
from keras.models import load_model
fashion_model = load_model('semTransfer.h5py')

In [ ]:
#Plotando graficos referentes a treinamento e validação acurácia e loss
accuracy = fashion_train_dropout.history['accuracy']
val_accuracy = fashion_train_dropout.history['val_accuracy']
loss = fashion_train_dropout.history['loss']
val_loss = fashion_train_dropout.history['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
#Predizendo a  classe a partir da base de teste_X
predicted_classes = fashion_model.predict(test_X)
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)
predicted_classes.shape, test_Y.shape

In [ ]:
#Mostrando a quantidade de acertos e alguns exemplos
correct = np.where(predicted_classes==test_Y)[0]
print ("Encontrado %d amostras corretas" % len(correct))
for i, correct in enumerate(correct[0:9]):
    plt.subplot(3,3,i+1)
    img = cv2.cvtColor(test_X[correct].reshape(128,128,3),cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    #plt.imshow(test_X[correct].reshape(128,128,3), cmap='gray', interpolation='none')
    plt.title("Predito {}, Classe {}".format(predicted_classes[correct], test_Y[correct]))
    plt.tight_layout()

In [ ]:
#Mostrando a quantidade de erros e alguns exemplos
incorrect = np.where(predicted_classes!=test_Y)[0]
print ("Encontrado %d amostras incorretas" % len(incorrect))
for i, incorrect in enumerate(incorrect[:9]):
    plt.subplot(3,3,i+1)
    img = cv2.cvtColor(test_X[incorrect].reshape(128,128,3),cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    #plt.imshow(test_X[incorrect].reshape(128,128,3), cmap='gray', interpolation='none')
    plt.title("Pred {} : Cl {}".format(predicted_classes[incorrect], test_Y[incorrect]))
    plt.tight_layout()

In [ ]:
#Precisão de classificação
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(test_Y, predicted_classes, target_names=target_names))